### **1.1. Get the list of Michelin restaurants**  
You should begin by compiling a list of restaurants to include in your document corpus. Specifically, you will focus on web scraping the Michelin Restaurants in Italy. Your task is to collect the URL associated with each restaurant in this list. The output of this step should be a .txt file where each line contains a single restaurant’s URL. By the end, you should have approximately 2,000 restaurants on your list. The number changes daily, so some groups might have different number of restaurants.

In [1]:
import requests 
from bs4 import BeautifulSoup 
import os
import requests
import time
import json
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm 
from folium import Marker
from folium import Map
import folium
from folium.plugins import HeatMap, Fullscreen
import numpy as np

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_restaurant_urls(base_url, num_pages=100):
    urls = set()  # Store unique URLs

    for page in range(1, num_pages + 1):
        page_url = f"{base_url}/page/{page}"
        response = requests.get(page_url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all <a> tags with class 'link'
            for link in soup.find_all('a', class_='link'):
                href = link.get('href')
                
                # Only add URLs containing "/restaurant/"
                if href and '/restaurant/' in href:
                    urls.add(f"https://guide.michelin.com{href}")
        else:
            print(f"Page {page} could not be loaded, please check.")  # Notify if a page fails to load

    return list(urls)

# Print and save URLs
urls = get_restaurant_urls('https://guide.michelin.com/en/it/restaurants', num_pages=100)

if urls:
    with open('restaurants_urls.txt', 'w') as file:
        for url in urls:
            file.write(f"{url}\n")
    print("Restaurant URLs saved successfully!")
else:
    print("No URLs found, please check the HTML structure.")  # Notify if no URLs are found



Restaurant URLs saved successfully!


### -1.2. Crawl Michelin restaurant pages
Once you have all the URLs on the list, you should:

Download the HTML corresponding to each of the collected URLs.
After collecting each page, immediately save its HTML in a file. This way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the restaurants from page 1, page 2, ... of the Michelin restaurant list.
Tip: Due to the large number of pages to download, consider using methods that can help shorten the process. If you employed a particular process or approach, kindly describe it.

In [25]:

# Read the URL list file
with open('restaurants_urls.txt', 'r') as file:
    urls = file.readlines()

# Function to create folders and download HTML files
def download_html_pages(urls, base_folder='michelin_pages'):
    # Create the base folder if it doesn't exist
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)

    for i, url in enumerate(urls):
        url = url.strip()
        page_number = (i // 20) + 1  # Each 20 restaurants belong to one page
        folder_path = os.path.join(base_folder, f'page_{page_number}')
        
        # Create the folder for each page if it doesn't exist
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        # Define the file path for each restaurant's HTML file
        file_path = os.path.join(folder_path, f'restaurant_{i+1}.html')
        
        # If the file already exists, skip downloading it
        if os.path.exists(file_path):
            print(f"{file_path} already exists, skipping.")
            continue

        try:
            # Download the HTML content of the restaurant page
            response = requests.get(url)
            if response.status_code == 200:
                with open(file_path, 'w', encoding='utf-8') as html_file:
                    html_file.write(response.text)
                print(f"{file_path} downloaded successfully.")
            else:
                print(f"{url} could not be downloaded, status code: {response.status_code}")
        except Exception as e:
            print(f"An error occurred while downloading {url}: {e}")
        
        time.sleep(1)  # Wait to prevent too fast downloads

# Download HTML pages
download_html_pages(urls)

michelin_pages/page_1/restaurant_1.html already exists, skipping.
michelin_pages/page_1/restaurant_2.html already exists, skipping.
michelin_pages/page_1/restaurant_3.html already exists, skipping.
michelin_pages/page_1/restaurant_4.html already exists, skipping.
michelin_pages/page_1/restaurant_5.html already exists, skipping.
michelin_pages/page_1/restaurant_6.html already exists, skipping.
michelin_pages/page_1/restaurant_7.html already exists, skipping.
michelin_pages/page_1/restaurant_8.html already exists, skipping.
michelin_pages/page_1/restaurant_9.html already exists, skipping.
michelin_pages/page_1/restaurant_10.html already exists, skipping.
michelin_pages/page_1/restaurant_11.html already exists, skipping.
michelin_pages/page_1/restaurant_12.html already exists, skipping.
michelin_pages/page_1/restaurant_13.html already exists, skipping.
michelin_pages/page_1/restaurant_14.html already exists, skipping.
michelin_pages/page_1/restaurant_15.html already exists, skipping.
mich

### -- 1.3 Parse downloaded pages
At this point, you should have all the HTML documents about the restaurant of interest, and you can start to extract specific information. The list of the information we desire for each restaurant and their format is as follows:

Restaurant Name (to save as restaurantName): string;
Address (to save as address): string;
City (to save as city): string;
Postal Code (to save as postalCode): string;
Country (to save as country): string;
Price Range (to save as priceRange): string;
Cuisine Type (to save as cuisineType): string;
Description (to save as description): string;
Facilities and Services (to save as facilitiesServices): list of strings;
Accepted Credit Cards (to save as creditCards): list of strings;
Phone Number (to save as phoneNumber): string;
URL to the Restaurant Page (to save as website): string.

In [26]:
def parse_restaurant_data(html_file_path):
    # Open and parse the HTML file
    with open(html_file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    
    # Find the <script> tag containing JSON data
    json_script = soup.find('script', type='application/ld+json')
    if not json_script:
        print(f"JSON data not found: {html_file_path}")
        return None
    
    # Parse JSON data
    restaurant_data = json.loads(json_script.string)
    
    # Extract required fields from JSON data
    data = {
        "restaurantName": restaurant_data.get("name", ""),
        "address": restaurant_data.get("address", {}).get("streetAddress", ""),
        "city": restaurant_data.get("address", {}).get("addressLocality", ""),
        "postalCode": restaurant_data.get("address", {}).get("postalCode", ""),
        "country": restaurant_data.get("address", {}).get("addressCountry", ""),
        # Extract 'priceRange' directly from JSON, if available
        "priceRange": restaurant_data.get("priceRange", ""),
        "cuisineType": restaurant_data.get("servesCuisine", ""),
        "description": restaurant_data.get("review", {}).get("description", ""),
        "facilitiesServices": ["Drive-through" if restaurant_data.get("hasDriveThroughService", False) == "True" else ""],
        "creditCards": restaurant_data.get("paymentAccepted", "").split(", "),
        "phoneNumber": restaurant_data.get("telephone", ""),
        "website": restaurant_data.get("url", "")
    }
    
    # If 'priceRange' is missing in JSON, search for it in the HTML using alternative class names
    if not data["priceRange"]:
        # Try alternative class names like "price-symbol", "price-range", "pricing-info", or "price"
        price_selectors = ["span.price-symbol", "span.price-range", "div.pricing-info", "div.price"]
        for selector in price_selectors:
            price_element = soup.select_one(selector)
            if price_element:
                data["priceRange"] = price_element.get_text(strip=True)
                break  # Stop the loop once a value is found
    
    # If 'priceRange' is still empty, search for any text containing the '€' symbol
    if not data["priceRange"]:
        possible_price_text = soup.find(string=lambda text: "€" in text if text else False)
        if possible_price_text:
            data["priceRange"] = possible_price_text.strip()

    # If 'priceRange' is still not found, display a message
    if not data["priceRange"]:
        print(f"'priceRange' data not found: {html_file_path}")
    
    return data
def save_to_tsv(data, output_folder, file_name):
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, file_name)
    
    # Write each piece of data in 'key=value' format to the TSV file
    with open(output_path, 'w', encoding='utf-8') as file:
        tsv_data = [
            f"restaurantName={data['restaurantName']}",
            f"address={data['address']}",
            f"city={data['city']}",
            f"postalCode={data['postalCode']}",
            f"country={data['country']}",
            f"priceRange={data['priceRange']}",
            f"cuisineType={data['cuisineType']}",
            f"description={data['description']}",
            f"facilitiesServices={'|'.join(data['facilitiesServices'])}",
            f"creditCards={'|'.join(data['creditCards'])}",
            f"phoneNumber={data['phoneNumber']}",
            f"website={data['website']}"
        ]
        file.write('\t'.join(tsv_data) + '\n')

# Define the main folder (e.g., 'michelin_pages')
main_folder = 'michelin_pages'
output_folder = 'tsv'  # Folder to save output TSV files
counter = 1

# Traverse all subfolders and HTML files
for root, dirs, files in os.walk(main_folder):
    for file_name in files:
        if file_name.endswith('.html'):
            file_path = os.path.join(root, file_name)
            data = parse_restaurant_data(file_path)
            if data:
                save_to_tsv(data, output_folder, f'restaurant_{counter}.tsv')
                counter += 1

print(f"All files processed. A total of {counter - 1} TSV files created.")


All files processed. A total of 1981 TSV files created.


In [48]:
import pandas as pd

# Path to the folder containing TSV files
tsv_folder = 'tsv'  # Assuming the files are saved in the 'tsv' folder
all_data = []  # Create an empty list to store all data
processed_files = 0  # Initialize a counter for processed files

# Read each TSV file and append its data to the list
for file_name in os.listdir(tsv_folder):
    if file_name.endswith('.tsv'):
        file_path = os.path.join(tsv_folder, file_name)
        
        try:
            # Read each TSV file line by line and parse key-value pairs
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                data_dict = {}
                for line in lines:
                    # Split each line into 'key=value' pairs and add to data_dict
                    parts = line.strip().split('\t')
                    for part in parts:
                        if '=' in part:  # Ensure there is an '=' character
                            key, value = part.split('=', 1)
                            data_dict[key] = value
                
                # Append the dictionary as a single row to a DataFrame
                all_data.append(pd.DataFrame([data_dict]))
                processed_files += 1  # Increment the processed files counter

        except Exception as e:
            print(f"An error occurred: {e} - File: {file_name}")

# Concatenate all DataFrames
try:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"All data combined successfully. Processed files: {processed_files}, Total rows: {len(combined_df)}")
except Exception as e:
    print(f"An error occurred during concatenation: {e}")

# Display the entire table (warning: may be too large if there are many rows)
combined_df


All data combined successfully. Processed files: 1983, Total rows: 1983


,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Osteria la Fefa,via Trento-Trieste 9/c,Finale Emilia,41034,ITA,€€,Emilian,"The name “Fefa” is a diminutive of Genoveffa, ...",,Credit card / Debit card accepted|Mastercard c...,+39 0535 780202,https://guide.michelin.com/en/emilia-romagna/f...
1,Campamac,strada della Valle 1,Barbaresco,12050,ITA,€€€,Piedmontese,A real gourmet inn which serves traditional Pi...,,American Express credit card|Credit card / Deb...,+39 0173 635051,https://guide.michelin.com/en/piemonte/barbare...
2,Lucenti,via Mazzini 38,Montefiorino,41045,ITA,€,Emilian,"This small, family - run restaurant furnished ...",,American Express credit card|Credit card / Deb...,+39 0536 965122,https://guide.michelin.com/en/emilia-romagna/m...
3,La Capanna di Eraclio,Località per Le Venezie 21,Codigoro,44021,ITA,€€€,Seafood,"Whenever we visit Capanna di Eraclio, we have ...",,American Express credit card|Credit card / Deb...,+39 0533 712154,https://guide.michelin.com/en/emilia-romagna/c...
4,Il Giardino del Gusto,piazza XX Settembre 6/c,Ventimiglia,18039,ITA,€€€,Creative,"Although not on the seafront, this restaurant ...",,American Express credit card|Credit card / Deb...,+39 0184 355244,https://guide.michelin.com/en/liguria/ventimig...
...,...,...,...,...,...,...,...,...,...,...,...,...
1978,Il Ristorante - Niko Romito,via Privata Fratelli Gabba 7/b,Milan,20121,ITA,€€€€,Italian Contemporary,Housed within the Bulgari hotel right in the h...,,American Express credit card|Credit card / Deb...,+39 02 805 8051,https://guide.michelin.com/en/lombardia/milano...
1979,Repubblica di Perno,"vicolo Cavour 5, loc. Perno",Monforte d'Alba,12065,ITA,€€,Piedmontese,Situated in the heart of the gastronomic Langa...,,Credit card / Debit card accepted|Mastercard c...,+39 0173 78492,https://guide.michelin.com/en/piemonte/monfort...
1980,Dry Aged,via Cesare Da Sesto 1,Milan,20123,ITA,€€,Contemporary,Two partners with significant experience opene...,,American Express credit card|Credit card / Deb...,+39 02 5810 7932,https://guide.michelin.com/en/lombardia/milano...
1981,Harry's Piccolo,piazza Unità d'Italia 2,Trieste,34121,ITA,€€€€,Italian Contemporary,"Thanks to its hushed ambience, Harry’s Piccolo...",,American Express credit card|Credit card / Deb...,+39 040 660606,https://guide.michelin.com/en/friuli-venezia-g...


## 2. Search Engine

This search engine allows you to retrieve restaurants based on a user query. We’ll build two types of search engines:

**Conjunctive Search Engine**: Returns restaurants where all query terms appear in the description.
**Ranked Search Engine**: Returns the top-k restaurants sorted by similarity to the query, using TF-IDF and Cosine Similarity.

#### Load Dataset

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
import heapq
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data = pd.read_csv("dataset.csv")
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,O Me O Il Mare,Via Roma 45/47,Gragnano,80054,Italy,€€€€,"Italian Contemporary, Modern Cuisine",After many years’ experience in Michelin-starr...,Air conditioning|Interesting wine list|Wheelch...,amex|dinersclub|mastercard|visa,+39 081 620 0550,http://omeoilmare.com
1,Alessandro Feo,via Angelo Lista 24,Marina di Casal Velino,84040,Italy,€€,"Campanian, Seafood",In a beautiful stone-vaulted building (an old ...,NaN,amex|dinersclub|mastercard|visa,+39 328 893 7083,https://www.alessandrofeoristorante.it/
2,Lalibera,via Pertinace 24/a,Alba,12051,Italy,€€,"Piedmontese, Traditional Cuisine","A modern, designer-style restaurant staffed by...",Air conditioning|Interesting wine list,amex|mastercard|visa,+39 0173 293155,https://lalibera.com/
3,Antica Macelleria Cecchini - Solociccia,via Chiantigiana 5,Panzano,50022,Italy,€€,Meats and Grills,One of the most famous butchers in Italy has n...,Air conditioning|Bring your own bottle|Restaur...,amex|mastercard|visa,+39 055 852020,https://www.dariocecchini.com
4,Brindo,via Libertà 18,Cusago,20047,Italy,€,Lombardian,The service is characterized by a cohesive and...,Air conditioning,amex|mastercard|visa,+39 02 9039 4429,https://www.brindo.it


In [5]:
data.isnull().sum()

restaurantName          0
address                 0
city                    0
postalCode              1
country                 0
priceRange              0
cuisineType             0
description             0
facilitiesServices     24
creditCards             3
phoneNumber             0
website               108
dtype: int64

In [6]:
# Replace null values in all columns with an empty string
data = data.fillna('')
data.isnull().sum()

restaurantName        0
address               0
city                  0
postalCode            0
country               0
priceRange            0
cuisineType           0
description           0
facilitiesServices    0
creditCards           0
phoneNumber           0
website               0
dtype: int64

In [7]:
data.shape

(1981, 12)

#### 2.0 Preprocessing

##### 2.0.0) Preprocessing text
Before building the search engine, you must clean and prepare the text in each restaurant’s description. We will:

Remove stopwords.
Remove punctuation.
Apply stemming.
Perform any other necessary cleaning to improve search accuracy.
For this, we use the nltk library.

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matteo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\matteo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\matteo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Replace non-standard apostrophes with standard ones
    text = text.replace("’", "'")
    # Remove punctuation
    no_punctuation = [char for char in text if char not in string.punctuation]
    text = ''.join(no_punctuation)

    # Tokenization (split the text into words)
    words = word_tokenize(text)

    # Remove stopwords in English
    stop_words = set(stopwords.words('english'))  # Stopwords for English
    filtered_words = [word for word in words if word not in stop_words]

    # Apply stemming using PorterStemmer
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    # Return the processed text as a string
    return ' '.join(stemmed_words)

In [10]:
# Ensure that all descriptions are strings and apply the preprocess function
data['cleaned_description'] = data['description'].apply(preprocess_text)

#### 2.1 Conjuctive Query
This first version of the search engine narrows the search to the description field of each restaurant. Only restaurants whose descriptions contain all the query words will be returned.

##### 2.1.1 Create Your Index!
**Vocabulary File**: Create a file called vocabulary.csv that maps each word to a unique integer (term_id).

**Inverted Index**: Build a dictionary mapping each term_id to a list of document IDs where that term appears.

In [11]:
import pandas as pd
from collections import defaultdict

# Step 1: Create the Vocabulary File

# 1. Tokenize the cleaned descriptions and create a set of unique words
unique_words = set()
for description in data['cleaned_description']:
    unique_words.update(description.split())

# 2. Create a vocabulary mapping word to term_id
vocabulary = {word: idx + 1 for idx, word in enumerate(sorted(unique_words))}

# Convert vocabulary to DataFrame and save as CSV
vocabulary_df = pd.DataFrame(list(vocabulary.items()), columns=['word', 'term_id'])
vocabulary_df.to_csv('vocabulary.csv', index=False)

# Step 2: Build the Inverted Index

# Initialize inverted index as a defaultdict
inverted_index = defaultdict(list)

# Iterate through each document (restaurant description)
for doc_id, description in enumerate(data['cleaned_description'], start=0):
    words_in_doc = set(description.split())  # Get unique words in the description
    for word in words_in_doc:
        if word in vocabulary:  # Ensure word exists in vocabulary
            term_id = vocabulary[word]
            inverted_index[term_id].append(doc_id)

# Save the inverted index to a file
import json
with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)

##### 2.1.2 Execute the Query

When the user inputs a query, for example, "modern seasonal cuisine", the search engine will:

Process the query terms.
Find and return a list of restaurants containing all query words in their description.

In [12]:
import json
import pandas as pd

with open('inverted_index.json', 'r') as file:
    inverted_index = json.load(file)

# Function to process the input query and find matching restaurants
def process_query(query):
    # Tokenize and clean the query
    query_terms = word_tokenize(preprocess_text(query))

    # Find term IDs for query words
    term_ids = []
    for term in query_terms:
        # Look for term in the inverted index and get corresponding term_id(s)
        # Here we assume that `inverted_index` maps term IDs to document IDs
        term_id = vocabulary_df[vocabulary_df['word']==term]['term_id'].iloc[0]  # Simple example, you'd map from term to term_id
        if str(term_id) in inverted_index.keys():
            term_ids.append(str(term_id))

    # Find restaurants that contain all query terms
    matching_restaurants = set(inverted_index[term_ids[0]]) if term_ids else set()
    for term_id in term_ids[1:]:
        matching_restaurants &= set(inverted_index[term_id])  # Set intersection

    # Get restaurant details for matching restaurant IDs
    results = []
    for idx in matching_restaurants:
        restaurant = data.iloc[idx]
        results.append({
            'restaurantName': restaurant['restaurantName'],
            'address': restaurant['address'],
            'description': restaurant['description'],
            'website': restaurant['website']
        })

    return results


In [13]:
# Example usage:
query = "modern seasonal cuisine"
matching_restaurants = process_query(query)

# Output the results
for result in matching_restaurants[:5]:
    print(f"Name: {result['restaurantName']}")
    print(f"Address: {result['address']}")
    print(f"Description: {result['description']}")
    print(f"Website: {result['website']}\n")

Name: Winter Garden Florence
Address: piazza Ognissanti 1
Description: Horse-drawn carriages once entered the old courtyard of the St Regis hotel, now converted into an elegant winter garden which also includes a cocktail bar with sofas and armchairs. Seasonality and local gems are fundamental pillars of the modern Mediterranean cuisine.
Website: https://www.wintergardenflorence.com/it/

Name: Retrobottega
Address: via della Stelletta 4
Description: Minimalist decor and clean lines characterise this restaurant decorated in dark tones, where the two owner-chefs have both worked in various Michelin-starred restaurants (and others) over the years. Modern cuisine which showcases seasonal ingredients in regional yet refined dishes.
Website: https://www.retro-bottega.com

Name: Locanda Solagna
Address: piazza I  Novembre 2
Description: Although this restaurant has been in business since the 1950s, it feels much more modern thanks to its updated appearance. Renowned for its excellent regional

#### 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity
For the second search engine, given a query, retrieve the top-k restaurants ranked by relevance to the query.

##### 2.2.1 Inverted Index with TF-IDF Scores
- **TF-IDF Scores**: Calculate TF-IDF scores for each term in each restaurant’s description.
- **Updated Inverted Index**: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.


Calculate **Term Frequencies (TF)**\
For each document (restaurant description), we count how frequently each term appears and normalize by the total word count.

In [14]:
from collections import Counter
import numpy as np

# Initialize a dictionary to store term frequencies for each document
tf_dict = {}

# Calculate term frequency for each document
for doc_id, description in enumerate(data['cleaned_description'], start=0):
    word_counts = Counter(description.split())  # Count word occurrences
    total_words = sum(word_counts.values())  # Total words in the document
    tf_dict[doc_id] = {word: count / total_words for word, count in word_counts.items()}


Calculate **Document Frequencies (DF)**\
Document Frequency measures how many documents each term appears in.

In [15]:
# Initialize a dictionary to count document frequencies
df_dict = Counter()

# Calculate document frequency for each term
for description in data['cleaned_description']:
    words_in_doc = set(description.split())  # Unique words in the document
    df_dict.update(words_in_doc)

# Total number of documents
N = len(data)

#df_dict

**Calculate TF-IDF Scores** \
For each document, compute TF-IDF for all terms.

In [16]:
tfidf_scores = {}

# Calculate TF-IDF for each document
for doc_id, term_frequencies in tf_dict.items():
    tfidf_scores[doc_id] = {}
    for term, tf in term_frequencies.items():
        idf = np.log(N / (df_dict[term] + 1))  # Add 1 to avoid division by zero
        tfidf_scores[doc_id][term] = tf * idf

#tfidf_scores


**Store TF-IDF Scores in a DataFrame**

In [17]:
# Flatten the TF-IDF dictionary into a list of records
tfidf_records = []

for doc_id, terms in tfidf_scores.items():
    for term, score in terms.items():
        tfidf_records.append({'doc_id': doc_id,'term_id':vocabulary[term], 'term': term, 'tf':tf_dict[doc_id][term], 'idf':np.log(N / (df_dict[term] + 1)), 'tfidf': score})

# Create a DataFrame
tfidf_df = pd.DataFrame(tfidf_records)

# Save to CSV
tfidf_df.to_csv('tfidf_scores.csv', index=False)

tfidf_df.head()


,doc_id,term_id,term,tf,idf,tfidf
0,0,4178,mani,0.022727,2.716160,0.061731
1,0,7715,year,0.022727,2.509953,0.057044
2,0,2590,experi,0.022727,2.371001,0.053886
3,0,4420,michelinstar,0.022727,4.007838,0.091087
4,0,5826,restaur,0.068182,0.184646,0.012590


##### 2.2.2 Execute the Ranked Query
For the ranked search engine:
- Process the query terms.
- Use Cosine Similarity to rank matching restaurants based on the TF-IDF vectors of the query and each document.
- Return the top-k results or all matching restaurants if fewer than k have non-zero similarity.

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to process the input query and retrieve the top-k restaurants ranked by relevance to the query
def ranked_query(query,k):
    # Tokenize and clean the query
    query_terms = word_tokenize(preprocess_text(query))
    # Calculate the TF for each term in the query
    term_freq = Counter(query_terms)
    query_tf = {term: freq / len(query_terms) for term, freq in term_freq.items()}

    # Initialize query vector with 0s
    query_vector = np.zeros(len(tfidf_df['term_id'].unique()))  # Size based on unique terms
    # Map the query terms to their term IDs and calculate TF-IDF for each query term
    for term, tf in query_tf.items():
        if term in vocabulary:
            term_id = vocabulary[term]-1  # Get the term ID from the vocabulary
            idf_value = tfidf_df[tfidf_df['term'] == term]['idf'].iloc[0]  # Get the IDF value
            query_vector[term_id] = tf * idf_value  # Compute TF-IDF score for the query

    # Step 3: Prepare the document vectors
    # We need to create document vectors based on the tfidf_df
    doc_vectors = {}
    for doc_id in tfidf_df['doc_id'].unique():
        doc_vector = np.zeros(len(vocabulary))  # Vector for the document
        for _, row in tfidf_df[tfidf_df['doc_id'] == doc_id].iterrows():
            term_id = row['term_id']-1
            doc_vector[term_id] = row['tfidf']  # Assign the TF-IDF score for the term in the document
        doc_vectors[doc_id] = doc_vector

    # Step 4: Compute Cosine Similarity between the query and each document
    similarities = {}
    for doc_id, doc_vector in doc_vectors.items():
        cosine_sim = cosine_similarity([query_vector], [doc_vector])[0][0]
        similarities[doc_id] = cosine_sim

    # Step 5: Sort by similarity and return top-k results
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    top_k_results = sorted_similarities[:k]

    results = []
    for doc_id, score in top_k_results:
        restaurant = data.iloc[doc_id]
        results.append({
        'restaurantName': restaurant['restaurantName'],
        'address': restaurant['address'],
        'description': restaurant['description'],
        'website': restaurant['website'],
        'similarity_score':score.round(4)
        })

    return  results


In [19]:
query = "modern seasonal cuisine"
result_query = ranked_query(query,3)
result_query

[{'restaurantName': 'Saur',
  'address': 'via Filippo Turati 8',
  'description': 'In a tiny rural village, this contemporary, almost minimalist-style restaurant serves modern cuisine with an emphasis on seasonal, regional produce.',
  'website': 'https://ristorantesaur.it',
  'similarity_score': np.float64(0.2428)},
 {'restaurantName': 'La Botte',
  'address': 'via Giuseppe Garibaldi 8',
  'description': 'A modern and welcoming contemporary bistro situated in the heart of Stresa’s historic centre. Run by an entire family, the restaurant serves modern and imaginative fish and meat dishes where the focus is always on seasonal ingredients. The interesting wine list also includes a selection of wines by the glass.',
  'website': 'http://www.trattorialabottestresa.it',
  'similarity_score': np.float64(0.2325)},
 {'restaurantName': 'Razzo',
  'address': 'via Andrea Doria 17/f',
  'description': 'A quiet restaurant with a relaxed, young and modern feel serving contemporary cuisine prepared f

In [20]:
# If you prefer to visualize like this
output = pd.DataFrame(result_query)
output.head()

,restaurantName,address,description,website,similarity_score
0,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, al...",https://ristorantesaur.it,0.2428
1,La Botte,via Giuseppe Garibaldi 8,A modern and welcoming contemporary bistro sit...,http://www.trattorialabottestresa.it,0.2325
2,Razzo,via Andrea Doria 17/f,"A quiet restaurant with a relaxed, young and m...",https://vadoarazzo.it/,0.2246


### **Question 3: Define a New Score!**

Define the New Scoring Function: We can calculate scores based on the description, cuisine match, facilities, and price range.

In [21]:
def calculate_custom_score(restaurant, query, tfidf_vectorizer, tfidf_matrix):
    # Description Match Score using TF-IDF
    query_tfidf = tfidf_vectorizer.transform([query])
    description_score = (query_tfidf * tfidf_matrix[restaurant['index']].T).toarray()[0][0]

    # Cuisine Match: Simple string matching
    cuisine_score = 1 if query.lower() in restaurant['cuisineType'].lower() else 0

    # Facilities and Services: Checking for matches
    facilities = ['Terrace', 'Air conditioning']
    facilities_score = sum(1 for facility in facilities if facility.lower() in restaurant['facilitiesServices'].lower())

    # Price Range: Assigning a higher score for affordability
    price_score = 1 if restaurant['priceRange'].lower() in ['low', 'affordable'] else 0

    # Combine scores with weights
    total_score = description_score * 0.5 + cuisine_score * 0.2 + facilities_score * 0.2 + price_score * 0.1
    return total_score

Apply Scoring Function: Used heapq to maintain the top-k restaurants based on the scores.

In [22]:
def rank_restaurants(data, query):
    # Initialize TF-IDF Vectorizer and fit on restaurant descriptions
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'])

    # Create a min-heap to store the top-k restaurants
    top_k = 10
    heap = []

    for index, restaurant in data.iterrows():
        restaurant['index'] = index  # Add an index for TF-IDF scoring
        score = calculate_custom_score(restaurant, query, tfidf_vectorizer, tfidf_matrix)
        if len(heap) < top_k:
            heapq.heappush(heap, (score, restaurant))
        else:
            heapq.heappushpop(heap, (score, restaurant))

    # Sort heap by score in descending order
    ranked_restaurants = sorted(heap, key=lambda x: x[0], reverse=True)
    return [(res[1]['restaurantName'], res[1]['address'], res[1]['description'], res[1]['website'], res[0]) for res in ranked_restaurants]

 Output and Comparison

In [23]:
query = "affordable Italian restaurant with a terrace"
ranked_results = rank_restaurants(data, query)

for restaurant in ranked_results:
    print(f"Name: {restaurant[0]}, Address: {restaurant[1]}, Score: {restaurant[4]}")

Name: Trattoria Pomposa - Al Re gras, Address: via Castel Maraldo 57, Score: 0.48475083133133545
Name: La Veranda, Address: via Regina 40, Score: 0.4572618026025641
Name: Caffè Dante Bistrot, Address: piazza dei Signori 2, Score: 0.4557231924723664
Name: ZELO, Address: via Gesù 6/8, Score: 0.45514629615116364
Name: Locanda di Nonna Ida, Address: Località Pontarola 12, Score: 0.45225511849806865
Name: Capogiro, Address: Località Li Mucchi Bianchi, Score: 0.4498920084616719
Name: La Perla del Mare, Address: via della Meloria 9, Score: 0.4496610371083053
Name: Io e Luna, Address: località Montebello 1, Score: 0.44727094626360364
Name: Il Ristorante - Niko Romito, Address: via di Ripetta 73, Score: 0.44629988630652917
Name: Osteria Il Cappello, Address: piazzetta Bruno Lunelli 5, Score: 0.44460824351577355


### COMPARISON :
The custom scoring function in part 3 is markedly more sophisticated and effective than the TF-IDF-only method used in part 2. By balancing textual and attribute-based matching, it significantly improves the user experience, making recommendations more aligned with practical, real-world preferences. The incorporation of factors like cuisine type, facilities, and price range allows the system to address the diverse criteria users may have when choosing a restaurant, ultimately enhancing the utility and appeal of the recommendation engine.

## **Question 4**

The postal code is used to uniquely identify the city where the restaurant is located, with preference given to this variable over the city name, as the latter can be influenced by variations or translation issues.

In [ ]:
combined_df[combined_df['postalCode'].isnull()]

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website


No missing data, so we can proceed.

Starting from these postal codes, I define a function that allows me to find the coordinates, which are expressed in latitude and longitude.

In [7]:
# Define a function to get the coordinates (latitude and longitude) based on a postal code
def get_coordinates(postal_code):
    # Initialize the geolocator with a user-agent
    geolocator = Nominatim(user_agent="postal_code_geocoder")
    
    try:
        # Try to get the location based on the postal code and country (Italy)
        location = geolocator.geocode({'postalcode': postal_code, 'country': 'Italy'})
        
        # If a location is found, return its latitude and longitude
        if location:
            return location.latitude, location.longitude
        else:
            # Return None for both latitude and longitude if no location is found
            return None, None
    
    except Exception as e:
        # Return None for both latitude and longitude if there's an error
        return None, None

Using the same logic, define a function that allows me to find the region where the restaurant is located

In [51]:
def get_region_from_postal_code(postal_code):
    geolocator = Nominatim(user_agent="postal_code_geocoder")
    try:
        # Attempt to get the location for the given postal code and country (Italy)
        location = geolocator.geocode({'postalcode': postal_code, 'country': 'Italy'}, addressdetails=True)
        
        if location and 'address' in location.raw:
            address = location.raw['address']
            # Extract the region from the address, defaulting to 'Regione non trovata' if not found
            regione = address.get('state', 'Regione non trovata')  
            return regione
        else:
            return "Regione non trovata"
    
    except Exception as e:
        # In case of an error, return the error message
        return f"Errore: {e}"


I will add these additional details to the original dataset for now.

In [22]:
# Add new columns for latitude and longitude
combined_df['latitude'] = None
combined_df['longitude'] = None

# Iterate through each postal code to update coordinates
for idx, postal_code in tqdm(combined_df['postalCode'].items(), total=len(combined_df)):
    latitude, longitude = get_coordinates(postal_code)
    
    # Update values directly in the DataFrame
    combined_df.at[idx, 'latitude'] = latitude
    combined_df.at[idx, 'longitude'] = longitude

100%|██████████| 1983/1983 [16:35<00:00,  1.99it/s]


In [52]:
combined_df['regione'] = combined_df['postalCode'].apply(get_region_from_postal_code)

In [54]:
combined_df[combined_df['regione'].isnull()]

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,regione


Additionally, I will manually specify the 4 postal codes that the function was unable to identify.

In [25]:
# Update the dataset for rows where the city is 'Modica'
combined_df.loc[combined_df['city'] == 'Modica', 'latitude'] = 36.8500
combined_df.loc[combined_df['city'] == 'Modica', 'longitude'] = 14.7667

combined_df.loc[combined_df['city'] == 'Syracuse', 'latitude'] = 37.0755
combined_df.loc[combined_df['city'] == 'Syracuse', 'longitude'] = 15.2866

# Update the dataset for rows where the city is 'Rotonda'
combined_df.loc[combined_df['city'] == 'Rotonda', 'latitude'] = 39.9742
combined_df.loc[combined_df['city'] == 'Rotonda', 'longitude'] = 15.7491

# Update the dataset for rows where the city is 'Castelbianco'
combined_df.loc[combined_df['city'] == 'Castelbianco', 'latitude'] = 44.1056
combined_df.loc[combined_df['city'] == 'Castelbianco', 'longitude'] = 8.0703



combined_df[combined_df['latitude'].isnull()]

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,latitude,longitude,regione


Starting from the price index expressed in '€', I will convert it into a numeric index so that operations can be performed on it.

In [26]:
# Define the price mapping
price_mapping = {'€': 1, '€€': 2, '€€€': 3, '€€€€': 4}

# Create a new column 'prezzi' in combined_df based on 'priceRange'
combined_df['prezzi'] = combined_df['priceRange'].replace(price_mapping)

C:\Users\matteo\AppData\Local\Temp\ipykernel_35716\2559523132.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['prezzi'] = combined_df['priceRange'].replace(price_mapping)


In [31]:
combined_df = pd.read_csv("combined_df.csv")

This is because I am creating a sub-dataset that combines the various restaurants in the same municipality for representation purposes on the map

In [27]:
# Aggregation based on city
aggregated_df = combined_df.groupby('city', as_index=False).agg({
    'latitude': 'first',  # Take the first latitude
    'longitude': 'first',  # Take the first longitude
    'prezzi': lambda x: round(x.mean())  # Calculate the mean of prices and round
})

A map has been generated using the 'folium' library. Specifically, this is an initial map that displays all the restaurants through a heatmap based on their price, as illustrated by the legend

In [28]:
# Create a map centered on Rome
map_obj = folium.Map(location=[41.9028, 12.4964], zoom_start=6)
map_obj.add_child(Fullscreen())

# Color mapping based on prices
color_map = {
    1: 'green',
    2: 'yellow',
    3: 'orange',
    4: 'red'
}

# Custom legend
legend_html = '''
<div style="position: fixed; 
          bottom: 50px; left: 50px; width: 150px; height: 160px; 
          background-color: white; border:2px solid grey; z-index:9999; font-size:14px; padding:10px;">
  <b>Price Legend</b><br>
  <i style="background-color: green; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €<br>
  <i style="background-color: yellow; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €€<br>
  <i style="background-color: orange; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €€€<br>
  <i style="background-color: red; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €€€€
</div>
'''
map_obj.get_root().html.add_child(folium.Element(legend_html))

# List for heatmap data
heat_data = []

# Loop to add markers and heatmap data
for _, row in combined_df.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        # Add data for heatmap
        intensity = row['prezzi']
        normalized_intensity = {
            1: 0.2,  # Green
            2: 0.4,  # Yellow
            3: 0.7,  # Orange
            4: 1.0   # Red
        }.get(intensity, 0.1)

        heat_data.append([row['latitude'], row['longitude'], normalized_intensity])

# Add heatmap to the map
HeatMap(
    heat_data,
    radius=15,
    blur=10,
    max_zoom=1,
    gradient={0.2: 'green', 0.4: 'yellow', 0.7: 'orange', 1.0: 'red'}
).add_to(map_obj)

# Display the map
map_obj

Once the best K restaurants are identified, their names are extracted

In [43]:
LISTA=[]
query = "adorable restaurant in the center of the city"
ranked_results = rank_restaurants(data, query)
for restaurant in ranked_results:
    LISTA.append(restaurant[0])

A second map is generated, showing only the restaurants selected from the search. In this case, they are represented with points on the map.

In [44]:
# Filter new_df to select only rows where "risto" is present in the list
filtered_df = combined_df[combined_df['restaurantName'].isin(LISTA)]

# Aggregation based on city for the filtered DataFrame
aggregated_df = filtered_df.groupby('city', as_index=False).agg({
    'latitude': 'first',  # Take the first latitude
    'longitude': 'first',  # Take the first longitude
    'postalCode': 'first',  # Take the first postal code
    'regione': 'first',  # Take the first region
    'prezzi': lambda x: round(x.mean()),  # Calculate the mean of prices and round
    'restaurantName': 'count'  # Count the number of restaurants in the city
})


# Create the map
map_obj = folium.Map(location=[41.9028, 12.4964], zoom_start=6)
map_obj.add_child(Fullscreen())

# Color map for prices
color_map = {
    1: 'green',
    2: 'yellow',
    3: 'orange',
    4: 'red'
}

# Add legend to the map
legend_html = '''
<div style="position: fixed; 
          bottom: 50px; left: 50px; width: 150px; height: 160px; 
          background-color: white; border:2px solid grey; z-index:9999; font-size:14px; padding:10px;">
  <b>Price Legend</b><br>
  <i style="background-color: green; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €<br>
  <i style="background-color: yellow; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €€<br>
  <i style="background-color: orange; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €€€<br>
  <i style="background-color: red; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> €€€€
</div>
'''
# Add legend to the map
map_obj.get_root().html.add_child(folium.Element(legend_html))

for _, row in aggregated_df.iterrows():
    color = color_map.get(row['prezzi'], 'blue')  # Color based on price

    # Description for the marker tooltip
    description = f"City: {row['city']}, Region: {row['regione']}, Restaurants: {row['restaurantName']}"

    # Add marker to the map
    Marker(
        location=(row['latitude'], row['longitude']),
        tooltip=description,
        icon=folium.Icon(color=color)
    ).add_to(map_obj)

# Display the map
map_obj

C:\Users\matteo\AppData\Local\Temp\ipykernel_25768\1958449040.py:52: UserWarning: color argument of Icon should be one of: {'orange', 'purple', 'gray', 'darkred', 'pink', 'red', 'lightgreen', 'blue', 'cadetblue', 'green', 'white', 'lightred', 'darkpurple', 'darkblue', 'beige', 'lightgray', 'lightblue', 'darkgreen', 'black'}.
  icon=folium.Icon(color=color)


## 5. BONUS: Advanced Search Engine

In [1]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
import pandas as pd

In [2]:
merged_data = pd.read_csv("merged_data.csv")
merged_data = merged_data.fillna('')
#merged_data.isnull().sum()

#### Build inverted indexes for restaurantName, city and cuisineType

In [3]:
import re
from collections import defaultdict

# Inverted index for restaurantName
inverted_index_rest = defaultdict(list)
for doc_id, rest in enumerate(merged_data['restaurantName'], start=0):
    words_in_rest = set(re.findall(r'\b\w+\b', rest.lower()))  # Get unique words in the description
    for word in words_in_rest:
        inverted_index_rest[word].append(doc_id)

# Inverted index for city
inverted_index_city = defaultdict(list)
for doc_id, c in enumerate(merged_data['city'], start=0):
    words_in_c = set(re.findall(r'\b\w+\b', c.lower()))  # Get unique words in the description
    for word in words_in_c:
        inverted_index_city[word].append(doc_id)

# Inverted index for cuisine
inverted_index_cuisine = defaultdict(list)
for doc_id, cuisine in enumerate(merged_data['cuisineType'], start=0):
    words_in_c = set(re.findall(r'\b\w+\b', cuisine.lower()))  # Get unique words in the description
    for word in words_in_c:
        inverted_index_cuisine[word].append(doc_id)

In [5]:
unique_creditCards = set()
for i in list(merged_data["creditCards"].unique()):
    single_creditcard = i.split('|')
    for x in single_creditcard:
        if x !='':
            unique_creditCards.add(x)
#unique_creditCards

facilitiesoptions = set()
for i in list(merged_data["facilitiesServices"].unique()):
    single_creditcard = i.split('|')
    for x in single_creditcard:
        if x !='':
            facilitiesoptions.add(x)
#facilitiesoptions

filtered_regions = merged_data["regione"].unique()
filtered_regions = [region for region in filtered_regions if region != "Regione non trovata"]

In [6]:
def advanced_search_engine(query):
        query_terms = query.lower().split()
        matched_restaurants = set()  # Collect matched restaurant IDs
        scores = {}  # Collect scores for matched restaurants

        for term in query_terms:
            # Match against restaurantName
            if term in inverted_index_rest:
                for rest_id in inverted_index_rest[term]:
                    matched_restaurants.add(rest_id)
                    scores[rest_id] = scores.get(rest_id, 0) + 3  # Weight = 3

            # Match against city
            if term in inverted_index_city:
                for rest_id in inverted_index_city[term]:
                    matched_restaurants.add(rest_id)
                    scores[rest_id] = scores.get(rest_id, 0) + 2  # Weight = 2

            # Match against cuisineType
            if term in inverted_index_cuisine:
                for rest_id in inverted_index_cuisine[term]:
                    matched_restaurants.add(rest_id)
                    scores[rest_id] = scores.get(rest_id, 0) + 1  # Weight = 1

        # Retain scores for filtered restaurants
        final_restaurants = [(rest_id, scores[rest_id]) for rest_id in matched_restaurants]

        # Sort by score (if needed)
        final_restaurants = sorted(final_restaurants, key=lambda x: x[1], reverse=True)
        return final_restaurants

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Restaurant Name Input
name_rest = widgets.Text(
    value='',
    placeholder='Type something',
    description='Restaurant:',
    disabled=False   
)

# City Name Input
name_city = widgets.Text(
    value='',
    placeholder='Type something',
    description='City:',
    disabled=False   
)

# Cuisine Type Input
name_cuisine = widgets.Text(
    value='',
    placeholder='Type something',
    description='Cuisine:',
    disabled=False   
)

# Price Range Slider
price_slider = widgets.SelectionRangeSlider(
    options=["€", "€€", "€€€", "€€€€"],
    index=(0, 3),
    description='Price Range',
    disabled=False
)

# Capture Price Range
def on_value_change(change):
    global selected_min_price, selected_max_price
    selected_min_price, selected_max_price = change['new']

price_slider.observe(on_value_change, names='value')


# Create Checkbox Grid
def create_checkbox_grid(options, num_columns):
    checkboxes = [widgets.Checkbox(value=False, description=opt) for opt in options]
    grid = widgets.GridBox(
        children=checkboxes,
        layout=widgets.Layout(
            grid_template_columns=f'repeat({num_columns}, 200px)',  
            grid_gap='10px',
        )
    )
    return checkboxes, grid

# Generate Grids for Credit Cards and Facilities
region_checkboxes, region_grid = create_checkbox_grid(filtered_regions, 3)
credit_card_checkboxes, credit_card_grid = create_checkbox_grid(unique_creditCards, 2)
facilities_checkboxes, facilities_grid = create_checkbox_grid(facilitiesoptions, 2)

# Get Selected Options
def get_selected_options(checkboxes):
    return [cb.description for cb in checkboxes if cb.value]

# Submit Button
button = widgets.Button(description="Search")

# Output Widget
output = widgets.Output()

# Handle Button Click
def on_button_click(b):
    with output:
        clear_output()
        selected_restaurant = name_rest.value.strip()
        selected_city = name_city.value.strip()
        selected_cuisine = name_cuisine.value.strip()
        selected_price_range = (selected_min_price, selected_max_price)
        selected_regions = get_selected_options(region_checkboxes)
        selected_credit_cards = get_selected_options(credit_card_checkboxes)
        selected_facilities = get_selected_options(facilities_checkboxes)
        query1 = selected_restaurant+' '+selected_city+' '+selected_cuisine

        output_query = advanced_search_engine(query1)
        filtered_restaurants = []
        for rest_id, score in output_query:
        # Retrieve row merged_data for the restaurant
            row = merged_data.iloc[rest_id]
            reg = row['regione']

            if reg not in selected_regions:
                continue
            
            # Check if the price in row falls within the selected price range
            if not (selected_min_price.count('€') <= row["priceRange"].count('€') <= selected_max_price.count('€')):
                continue
            
            # Check if all selected credit cards are present in the row's credit cards
            credit_cards = set(row["creditCards"].split('|'))
            if not all(card in credit_cards for card in selected_credit_cards):
                continue
            
            # Check if all selected facilities are present in the row's facilities
            facilities = set(row["facilitiesServices"].split('|'))
            if not all(facility in facilities for facility in selected_facilities):
                continue
            
            # If all conditions are satisfied, add the restaurant to the filtered list
            filtered_restaurants.append(rest_id)
        for r_id in filtered_restaurants:
            print("Restaurant Name: ", merged_data['restaurantName'].iloc[r_id])
            print("Address: ", merged_data['address'].iloc[r_id])
            print("Cuisine Type: ", merged_data['cuisineType'].iloc[r_id])
            print("Price Range: ", merged_data['priceRange'].iloc[r_id])
            print("Website: ", merged_data['website'].iloc[r_id])

button.on_click(on_button_click)


In [8]:
from ipywidgets import VBox, HBox, Label, Accordion, Layout

# Search Criteria Section
search_criteria_box = VBox(
    [
        Label("Search Criteria", style={"font-weight": "bold", "font-size": "16px"}),
        name_rest,
        name_city,
        name_cuisine,
        price_slider,
    ],
    layout=Layout(padding="10px", border="1px solid black", margin="10px")
)

# Filters Section
filters_box = VBox(
    [
        Label("Filter Options", style={"font-weight": "bold", "font-size": "16px"}),
        Label("Region:"),
        region_grid,
        Label("Accepted Credit Cards:"),
        credit_card_grid,
        Label("Facilities:"),
        facilities_grid,
    ],
    layout=Layout(padding="10px", border="1px solid black", margin="10px")
)

# Collapsible Filters
accordion = Accordion(
    children=[filters_box],
    titles=("Filters",),
    layout=Layout(margin="10px")
)
accordion.set_title(0, "Filter Options")

# Submit Section
submit_section = VBox(
    [button],
    layout=Layout(padding="10px", margin="10px", justify_content="center")
)

# Output Section
output_section = VBox(
    [output],
    layout=Layout(padding="10px", border="1px solid black", margin="10px")
)

# Full Layout
layout = VBox(
    [
        search_criteria_box,
        accordion,
        submit_section,
        output_section
    ],
    layout=Layout(padding="10px", margin="20px")
)

# Display the Interface
display(layout)


# ALGORİTHMİC QUESTİON

1.

Input: t - number of test cases

For each test case:
    1. Read n - number of packages.
    2. Initialize an empty list `packages`.
    3. For each package (i from 1 to n):
       - Read coordinates (x_i, y_i) and add (x_i, y_i) to `packages`.
    4. Sort `packages` by x-coordinate, then by y-coordinate.
    
    5. Initialize current position as (0, 0).
    6. Initialize an empty string `path`.
    7. Set `possible` to True.

    8. For each package in sorted `packages`:
       - Calculate the horizontal moves required to reach the package's x-coordinate from the current x-coordinate.
       - Calculate the vertical moves required to reach the package's y-coordinate from the current y-coordinate.
       - If the target x-coordinate is less than the current x-coordinate or 
         the target y-coordinate is less than the current y-coordinate:
           - Set `possible` to False and break the loop (the package is unreachable).
       - Otherwise:
           - Append 'R' repeated (target x - current x) times to `path` for right moves.
           - Append 'U' repeated (target y - current y) times to `path` for up moves.
           - Update current position to the package's coordinates.

    9. If `possible` is True:
       - Append "YES" and `path` to the output.
    10. If `possible` is False:
       - Append "NO" to the output.

Output: For each test case, print "YES" and the path if possible, or "NO" if not.





In [15]:
# Get all input at once
input_data = input("Enter the input: ").splitlines()  # Girdiyi bir seferde al ve satır satır ayır

# Parse the number of test cases
t = int(input_data[0])  # İlk satır test vaka sayısı

index = 1  # Başlangıç indexi

# For each test case
for _ in range(t):
    # Read the number of packages
    n = int(input_data[index])  # Paket sayısını oku
    index += 1
    
    packages = []
    
    # Read the coordinates of each package
    for _ in range(n):
        x, y = map(int, input_data[index].split())  # Her paketin koordinatlarını oku
        packages.append((x, y))
        index += 1
    
    # Sort the packages by x and y coordinates
    packages.sort()  # Paketleri x ve y koordinatlarına göre sırala
    
    # Initialize starting position and other variables
    current_x, current_y = 0, 0
    path = ""
    possible = True
    
    # Iterate over sorted packages
    for x, y in packages:
        # Check if package is reachable
        if x < current_x or y < current_y:
            possible = False
            break
        else:
            # Add right movements ('R') to the path string
            path += 'R' * (x - current_x)
            # Add up movements ('U') to the path string
            path += 'U' * (y - current_y)
            # Update the current position
            current_x, current_y = x, y
    
    # Output result
    if possible:
        print("YES")
        print(path)
    else:
        print("NO")

YES
RUUURRRRUU
NO
YES
RRRRUUU





3.
To analyze the time complexity of this algorithm, let’s break down each step and calculate its complexity. This will allow us to express the total complexity in Big-O notation.

Step-by-Step Analysis and Time Complexity

	1.	Input Reading:
	•	The algorithm reads the number of test cases, t, and for each test case, it reads the coordinates of n packages.
	•	Reading the input for n packages takes O(n) time per test case. With t test cases, this step contributes O(n) time for each case.
	2.	Sorting Step:
	•	The packages are sorted based on their  (x, y)  coordinates, which typically takes O(n \log n) time.
	•	Since sorting is the most time-consuming step in the algorithm, it contributes the largest portion of the time complexity. As n grows, this sorting step scales with O(n \log n).
	3.	Path Construction:
	•	After sorting, the algorithm iterates over each package, calculates the necessary right (R) and up (U) moves, and appends them to the path string.
	•	This step takes O(n) time, as each package is processed once.
	4.	Generating Output:
	•	Finally, the algorithm prints “YES” or “NO” and the path string for each test case.
	•	Output generation takes O(n) time because the path string is built based on the coordinates of the n packages.

Overall Time Complexity

Since sorting is the most time-intensive step for each test case, the total time complexity for a single test case is O(n \log n).

With t test cases, the overall time complexity becomes:


O(t \cdot n \log n)


Steps Contributing Most to Complexity

	1.	Sorting Step (O(n \log n)): Sorting the packages by x and y coordinates is the primary contributor to the algorithm’s time complexity because, as n increases, the additional  \log n  factor from sorting has a significant impact.
	2.	Path Construction Step (O(n)): Calculating the movements and constructing the path string takes O(n) time, but its impact is smaller compared to sorting.

Therefore, the overall time complexity of the algorithm is O(t \cdot n \log n), with the sorting step being the dominant factor.


4.  Assessment of Time Complexity Using LLM
 If we were to ask  CLAUDE  to evaluate the time complexity,

Let's analyze the main parts of the code:

Input handling and parsing:
O(1) - constant time

For each test case:

Reading coordinates for n packages: O(n)
Sorting packages: O(n log n)
Creating route for each package (inner loop): O(n)

Overall time complexity analysis:

For each test case, the dominant term is O(n log n) due to the sorting operation
With t test cases, the total complexity is: O(t * n log n)

This assessment is accurate because:

Python's sort() method uses TimSort algorithm which has O(n log n) complexity
Other operations (route creation, coordinate reading) are O(n)
Since O(n log n) > O(n), the dominant term for each test case is O(n log n)

In worst-case scenario:

Route calculation for each package: O(n)
Sorting operation: O(n log n)
t test cases: O(t)

Therefore, O(t * n log n) is the correct time complexity.
Potential improvement:

If the order of packages input doesn't matter specifically, removing the sorting operation could reduce the complexity to O(t * n).

5.
The greedy algorithm, which directs the robot to the closest package from its current location, is not necessarily optimal in terms of minimizing the total distance traveled. While it may seem intuitive to go to the closest package first, this approach does not guarantee the shortest possible total path. Here’s why:

Counterexample Showing the Greedy Algorithm is Not Optimal

Consider the following setup:
	1.	Robot starting position: (0, 0)
	2.	Package locations:
	•	Package 1 at (3, 0)
	•	Package 2 at (3, 3)
	•	Package 3 at (0, 3)

Greedy Approach (Closest-First)

Using the greedy approach, the robot will:
	1.	Move from (0, 0) to the closest package, Package 1 at (3, 0), traveling a distance of 3.
	2.	From (3, 0), the closest package is Package 2 at (3, 3), traveling an additional distance of 3.
	3.	From (3, 3), the last package (Package 3 at (0, 3)) is reached, traveling another distance of 3.

The total distance traveled using the greedy approach is 3 + 3 + 3 = 9.

Optimal Approach

To minimize the total distance, a better approach would be:
	1.	Move from (0, 0) to Package 3 at (0, 3), traveling a distance of 3.
	2.	From (0, 3), move to Package 2 at (3, 3), traveling an additional distance of 3.
	3.	From (3, 3), move to Package 1 at (3, 0), traveling a distance of 3.

The total distance traveled with this order is also 3 + 3 + 3 = 9.

However, in cases where there are more than three points or different orientations, such as a zigzag or non-straight path, this can be suboptimal
